In [1]:
# Import app
from celery_basics import app
from celery_basics import add
from celery_basics import make_pi
from celery_basics import apply_function
from celery_basics import apply_counter

#### Sum 1 to 2

In [2]:
res = add.apply_async(args=(1, 2))
results = res.get()
results

3

#### Calculate pi in range 1 to 10**5

In [3]:
%%timeit
from celery import group
job = group(make_pi.subtask((10**x, )) for x in range(0,8))
job_result = job.delay()
results = job_result.get()
print(results)

[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
1 loop, best of 3: 5.57 s per loop


In [4]:
%%timeit
pis = []
for x in range(0,8):
    pis.append(make_pi(10**x))
print(pis)

[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
1 loop, best of 3: 5.77 s per loop


#### Sum the numbers between the 1 to 1000 twice

In [5]:
%%timeit
from celery import chain
jobs = chain(add.si(i, i) for i in range(1,1001))
job_result = jobs.apply_async()
results = job_result.get()
print(results)

2000
2000
2000
2000
1 loop, best of 3: 26.9 s per loop


In [6]:
%%timeit
results = sum([add(i, i) for i in range(1,1001)])
print(results)

1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
10 loops, best of 3: 17.9 ms per loop


#### Counter of the words in reviews summaries

In [7]:
import pandas as pd

In [8]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8", iterator=True, chunksize=1000)

In [9]:
%%timeit
from collections import Counter
counter = Counter()
for review in reviews:
    counter += review.summary.apply(lambda x :Counter(str(x).split(" "))).values.sum()
counter.most_common()

The slowest run took 11749667.04 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 11.9 µs per loop


#### Counter of the words in reviews summaries with Celery

In [ ]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8", iterator=True, chunksize=100)

In [ ]:
%%timeit
import pickle
jobs =[]
for review in reviews:
    job = apply_counter.apply_async(args=(pickle.dumps(review),))
    jobs.append(job)
for job in jobs:
    counter += pickle.loads(job.get()) 

In [ ]:
counter.sum().most_common()